# Step 1: configuration

Input your walletPrivateKey here:

In [1]:
# make sure you configure mindlakesdk, please check env.py in the same folder
# please familar these configure, and you can read https://github.com/mind-network/mind-lake-sdk-python 
import env

print("check env.walletPrivateKey:", env.walletPrivateKey[0:5])
print("check env.mindLakeAppKey:", env.mindLakeAppKey[0:5])
print("check env.MINDLAKE_GATEWAY:", env.MINDLAKE_GATEWAY)

check env.walletPrivateKey: 2a776
check env.mindLakeAppKey: Kyunq
check env.MINDLAKE_GATEWAY: https://sdk.mindnetwork.xyz/node


Install depedancy and source code

In [2]:
# please "pip install mindlakesdk" if not installed
import mindlakesdk

# please "pip install minddatapack" if not installed, or git clone the source code
from minddatapack import DataPack

# please "pip install pandas" if not installed, it is used to display structure content in the notebook
import pandas

In [3]:
# check if example_data.csv file exists in the same folder, this demo will start to load this csv file.

# check if arweave_wallet.json file exists in the same folder, this demo needs to use arweave wallet to sign to send tx.

## Step 2. Local CSV -> MindLake
- Load data from a local CSV file without the metadata file, by defining each column manually.
- You can also load from a local backup into MindLake once you are familar, then no need to define each column. 

In [4]:
fn_local_csv = './example_data.csv'

# preview local csv
df = pandas.read_csv(fn_local_csv)
df

,uid,wallet_address,register_date
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


In [5]:
dataPack1 = DataPack(env.walletPrivateKey)
result = dataPack1.loadFromCSVFileByDefineColumn(fn_local_csv, 
    [
        DataPack.Column('uid', DataPack.DataType.int4, False),
        DataPack.Column('wallet_address', DataPack.DataType.text, True),
        DataPack.Column('register_date', DataPack.DataType.timestamp, True)
    ])
assert result, result.message
df = pandas.DataFrame(dataPack1.data)
df

,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into a table in MindLake

In [6]:
mindlake = mindlakesdk.connect(env.walletPrivateKey, env.mindLakeAppKey, env.MINDLAKE_GATEWAY)
assert mindlake, mindlake.message

# drop the table if exists
result = mindlake.datalake.dropTable('a_new_table')
print("dropTable result:", result.message)

# upload from local csv into mindlake
result = dataPack1.saveToMindLake('a_new_table', mindlake)
assert result, result.message
print("a_new_table has been saved to MindLake.")

dropTable result: Success
a_new_table has been saved to MindLake.


# you can also check this new downloaded table in https://scan.mindnetwork.xyz/account/myData

## Step 3. MindLake -> Arweave
- Load data from a table in MindLake and save to Arweave

In [7]:
dataPack2 = DataPack(env.walletPrivateKey)
# make sure if you complete Step 1, so "a_new_table" exists in mindlake 
result = dataPack2.loadFromMindByQuery('select * from "a_new_table"', mindlake)
assert result, result.message
df = pandas.DataFrame(dataPack2.data)
df

,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into Arweave

In [8]:
# make sure you have arweave_wallet.json 
result = dataPack2.saveToArweave('test_table_encrypted.csv', 'AR', './arweave_wallet.json')
assert result, result.message
print("test_table_encrypted.csv has been saved to Arweave.")
itemID = result.data
print("The Arweave ID is:", itemID)
arweave_explore_url = "https://viewblock.io/arweave/tx/%s"%(itemID)
arweave_file_url = "https://arseed.web3infra.dev/%s"%(itemID)
print("You can check on Arweave:", arweave_explore_url)
print("And the content on Arweave:", arweave_file_url)
import requests
content = requests.get(arweave_file_url).text
print("The content is:")
print(content)

test_table_encrypted.csv has been saved to Arweave.
The Arweave ID is: NNgcwmyBsB0F6sJGbEBcNxor7-i_5PF5LxcC2yOmFkU
You can check on Arweave: https://viewblock.io/arweave/tx/NNgcwmyBsB0F6sJGbEBcNxor7-i_5PF5LxcC2yOmFkU
And the content on Arweave: https://arseed.web3infra.dev/NNgcwmyBsB0F6sJGbEBcNxor7-i_5PF5LxcC2yOmFkU
The content is:
uid,wallet_address,register_date
123,\xc5c23a94d68b6159e0db4abfc83caa14012fdd2349e22a50b7800fa8a9da7c0dadcffc5a91ecc1b264922678a1aff26ce31a33b59ee0ca5236e5ddbae7aa15471b,\x62caa32d48c0d9e1c9411010de2eee60000ddbae901808bddbac1c54abb468d7ba
124,\x09b0a3fc52a0c0932e61b1c8c56b0bf7ee2af9e88e32b0663af2e03f50cbf65aee203a3df8ea5b15911cc629653ee70239e0c7f64fb4ebc7fc34f239ecc13736e5,\xaa53d8fad9156a1f54fffd7c329b871484962b59777bddbe970d5bfee00b344ed3
125,\x461ec6dc1cdf0edfaa1f44bfeeebd2bea56600b1d0f406dc5d5b781894a9511ef2108d1cf7cbb1117ca6027715beb60076a03920308855208f0d235c70d751cf74,\xa3bc165c1a5e6938b3e0596b87822ddbcc4c671292313c08963a05bfd1f6579147



## Step 4. Arweave -> Local File
- Load data from Arweave and save to local file

In [9]:
dataPack3 = DataPack(env.walletPrivateKey)
result = dataPack3.loadFromArweave(itemID)
assert result, result.message
print("You can see file on Arweave are encrypted, but datapack can decrypted it locally")
df = pandas.DataFrame(dataPack3.data)
df

You can see file on Arweave are encrypted, but datapack can decrypted it locally


,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into a local file with encryption

In [10]:
result = dataPack3.saveToLocalFile('test_localtable_encrypted.csv', False)
assert result, result.message
print("test_localtable_encrypted.csv has been saved to local file.")
df = pandas.read_csv('test_localtable_encrypted.csv')
df

test_localtable_encrypted.csv has been saved to local file.


,uid,wallet_address,register_date
0,123,\xbccbf2b5f37fa48a28cb629cdef5ce3a55485aaa1135...,\x92c68f85c32aaab96f93775287b998a263dd9ae413d9...
1,124,\x1772d0b504ceff425734fb9c0453466b2b0475de66d6...,\x591346e00096feb0823f3a5ce45d39fc2c0177a370ef...
2,125,\xc1d0276b43e14c9e912d0def9e7fbe4b2843650bc449...,\xe4090c2c346ab00d0923e8897a7f4b61a076112e7a3e...
